<a href="https://colab.research.google.com/github/hasibzunair/learn-pytorch/blob/master/notebooks/intro/Tutorial5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 5
Adapted from https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ || 
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data. 
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to 
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. 
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.




In [2]:
import os
import torch
from torch import nn # (all building blocks to build NNs!)
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU, 
if it is available. Let's check to see if 
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we 
continue to use the CPU. 



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and 
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method. 



In [4]:
# The new class inherits from nn.Module

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU() #10-dimensional tensor with raw values, not softmax
        )

    def forward(self, x):
        # operations applied to the input data are defined here
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print 
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_. 
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # raw values
pred_probab = nn.Softmax(dim=1)(logits) # apply sotfmax
y_pred = pred_probab.argmax(1) # similar to np.argmax
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


--------------




Model Layers
-------------------------

Lets break down the layers in the FashionMNIST model. To illustrate it, we 
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as 
we pass it through the network. 



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_ 
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear 
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # y = Wx + b, W -> weights, x-> input data
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
hidden1[0]

tensor([ 0.0050,  0.2405,  0.3159,  0.0608,  0.4854, -0.0260, -0.4059,  0.3408,
         0.6515,  0.1884, -0.1161,  0.3447,  0.6009, -0.0368,  0.3247,  0.3727,
         0.5872,  0.1978,  0.6001, -0.4182], grad_fn=<SelectBackward>)

In [13]:
hidden1[1]

tensor([-0.0358, -0.3439,  0.0920,  0.1167,  0.4405,  0.1749, -0.1845,  0.4905,
         0.5662,  0.1917,  0.1808,  0.0423,  0.5050, -0.0238,  0.3143,  0.0465,
         0.1995, -0.1190,  0.6662, -0.3829], grad_fn=<SelectBackward>)

nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [14]:
#Non-linear activations are what create the complex mappings 
#between the model's inputs and outputs. They are applied 
#after linear transformations to introduce nonlinearity,
#helping neural networks learn a wide variety of phenomena

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0050,  0.2405,  0.3159,  0.0608,  0.4854, -0.0260, -0.4059,  0.3408,
          0.6515,  0.1884, -0.1161,  0.3447,  0.6009, -0.0368,  0.3247,  0.3727,
          0.5872,  0.1978,  0.6001, -0.4182],
        [-0.0358, -0.3439,  0.0920,  0.1167,  0.4405,  0.1749, -0.1845,  0.4905,
          0.5662,  0.1917,  0.1808,  0.0423,  0.5050, -0.0238,  0.3143,  0.0465,
          0.1995, -0.1190,  0.6662, -0.3829],
        [ 0.0343,  0.2549,  0.4607,  0.3226,  0.5278,  0.1955, -0.1284,  0.4206,
          0.4972,  0.1426, -0.1465,  0.1922,  0.4631, -0.3976,  0.2665,  0.2616,
          0.2392,  0.3155,  0.5532, -0.5374]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0050, 0.2405, 0.3159, 0.0608, 0.4854, 0.0000, 0.0000, 0.3408, 0.6515,
         0.1884, 0.0000, 0.3447, 0.6009, 0.0000, 0.3247, 0.3727, 0.5872, 0.1978,
         0.6001, 0.0000],
        [0.0000, 0.0000, 0.0920, 0.1167, 0.4405, 0.1749, 0.0000, 0.4905, 0.5662,
         0.1917, 0.1808, 0.0423, 0.5050, 0.0000, 0.314

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered 
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [15]:
# nn.Sequential is a box which consists of different kinds of
# sushi in a pre-defined order. You can eat them only in that specific order! 
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28) # 3 images of size W and H.
logits = seq_modules(input_image) # forward pass
print(logits)

tensor([[ 0.0078, -0.1192,  0.3115,  0.1214, -0.2342,  0.3748,  0.3688,  0.1460,
          0.0581, -0.0054],
        [ 0.0742, -0.0522,  0.1611,  0.1451, -0.2688,  0.1163,  0.3995,  0.2041,
         -0.1086, -0.0234],
        [-0.0533, -0.1047,  0.3468,  0.1982, -0.4427,  0.1602,  0.3256,  0.1931,
          0.0127,  0.1476]], grad_fn=<AddmmBackward>)


nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values 
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along 
which the values must sum to 1. 



In [16]:
softmax = nn.Softmax(dim=1) # get PROBABILITY scores, scaled to [0, 1] by softmax operation
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights 
and biases that are optimized during training. Subclassing ``nn.Module`` automatically 
tracks all fields defined inside your model object, and makes all parameters 
accessible using your model's ``parameters()`` or ``named_parameters()`` methods. 

In this example, we iterate over each parameter, and print its size and a preview of its values. 




In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0234, -0.0127, -0.0024,  ..., -0.0253,  0.0211, -0.0005],
        [-0.0068, -0.0166,  0.0081,  ...,  0.0341, -0.0226,  0.0045]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0056, -0.0035], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0205, -0.0330,  0.0178,  ..., -0.0147,  0.0111, -0.0331],
        [-0.0129, -0.0193,  0.0115,  ..., -0.0331,  0.0250,  0.0383]],
       device='cuda

--------------




Further Reading
--------------
- `torch.nn API <https://pytorch.org/docs/stable/nn.html>`_

